In [9]:
from flask import Flask, request, jsonify, render_template, session
import pandas as pd
from flask_session import Session
import sys
sys.path.insert(0,'../')
from environment import env
config = env.env()

from model_functions.spacy_model import *
from model_functions.tfidf_model import *
from model_functions.nltk_model import *
from model_functions.gpt_model_functions import *
from model_functions.get_context import *
from lang_chatbot import nps_chain

In [10]:
#VARIABLES
#Select which model to use with chatbot
model = tfidf_model
OPENAI_API_KEY = config['gpt_api_key']

NameError: name 'nltk_model' is not defined

In [5]:
app = Flask(__name__)

# Configure server-side session
app.config['SESSION_TYPE'] = 'filesystem'
app.config['SECRET_KEY'] = 'supersecretkey'
Session(app)

@app.route('/reset', methods=['POST'])
def reset():
    session.clear()
    return jsonify({"response": "Session reset successful."})

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    # *Chat GPT assisted with session logic
    user_input = request.json.get("message")
    if not user_input:
        return jsonify({"response": "Please provide a message."})

    if 'stage' not in session:
        session['stage'] = 'initial'

    if session['stage'] == 'initial':
        # Initial stage: Handle general queries
        output, session['stage'], session['data'] = handle_initial_query(user_input)
        
    else:
        output = "I specialize only in queries related to amenities, events, alerts, park fees, park locations, and park descriptions. Please clarify your question."
        session['stage'] = 'initial'

    return jsonify({"response": output})

def handle_initial_query(user_input):
    # Simulate API call to get responses for the 'amenities' endpoint
    try: 
        output = nps_chain.invoke({"context": get_context(user_input, model), "question": user_input})
       
        return output, 'initial', None
    # If the bot does not understand the question, ask to clarify.
    except:
        return "I specialize only in queries related to amenities, events, alerts, park fees, park locations, and park descriptions. Please clarify your question.", 'initial', None


if __name__ == "__main__":
    app.run(port=8000)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [24/Jul/2024 15:27:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 15:28:03] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 15:28:13] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 15:28:31] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 15:28:55] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2024 15:29:30] "POST /chat HTTP/1.1" 200 -
